In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pyriemann.spatialfilters import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from IPython.display import clear_output
from scipy.stats import zscore
from tqdm import tqdm
from rcnet import rc_activations

rng = np.random.default_rng(0)

In [2]:
nsubs = 163
nsamples = 191
nchannels = 139

dynamics_length = 20
approx_reservoir_size = nchannels*2
worker_id = 0
w_in_init = "first_word_full_cleaning"
normalization = "zscore"
nfilters = 5
nsplits = 10


leak = 0.9854109308796132
degree = 19
radius = 0.28380506555016005

activations = np.zeros([nsubs, approx_reservoir_size, nsamples-dynamics_length-1])
for i in tqdm(range(1, 98), desc="HC"):
    name = 'filtered_data/HC_'+str(i)+'_det.dat'
    signal = np.loadtxt(name)
    activation = rc_activations(
        signal,
        dynamics_length,
        approx_reservoir_size,
        degree,
        radius,
        worker_id,
        w_in_init,
        normalization,
        leak
    )
    activations[i-1] = activation.T
for i in tqdm(range(98, 164), desc="MDD"):
    name = 'filtered_data/mdd_'+str(i)+'_det.dat'
    signal = np.loadtxt(name)
    activation = rc_activations(
        signal,
        dynamics_length,
        approx_reservoir_size,
        degree,
        radius,
        worker_id,
        w_in_init,
        normalization,
        leak
    )
    activations[i-1] = activation.T

covs = np.zeros([nsubs, approx_reservoir_size, approx_reservoir_size])
labels = np.zeros(nsubs)
for i in range(1, 98):
    activation = activations[i-1]
    covs[i-1] = np.cov(activation)
    labels[i-1] = 0
for i in range(98, 164):
    activation = activations[i-1]
    covs[i-1] = np.cov(activation)
    labels[i-1] = 1

confusion_matrices = np.zeros([nsplits, 2, 2])
split_i = 0
model = CSP(nfilter=nfilters, metric="euclid", log=True)
crossval = StratifiedShuffleSplit(n_splits=nsplits, test_size=0.2, random_state=0)
for train_index, test_index in tqdm(crossval.split(X=np.zeros(nsubs), y=labels), desc="CV", total=nsplits):
    model = model.fit(X=covs[train_index,:,:], y=labels[train_index])
    filtered_signals = model.transform(covs)
    lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
    clf = lda.fit(filtered_signals[train_index, :nfilters], labels[train_index])
    truths = labels[test_index]
    predictions = clf.predict(filtered_signals[test_index])
    confusion_matrices[split_i] = confusion_matrix(y_true=truths, y_pred=predictions)
    split_i += 1

CV: 100%|██████████| 10/10 [00:02<00:00,  4.92it/s]


In [3]:
tn = confusion_matrices[:, 0, 0]
fn = confusion_matrices[:, 1, 0]
tp = confusion_matrices[:, 1, 1]
fp = confusion_matrices[:, 0, 1]
accuracy = (tp+tn) / (tp+tn+fp+fn)
recall = tp / (tp+fn)
precision = tp / (tp+fp)
f1 = 2 * (
    (precision*recall) / (precision+recall)
)

In [4]:
print(f"Accuracy: {np.mean(accuracy):.2f} ± {np.std(accuracy):.2f}")
print(f"Recall: {np.mean(recall):.2f} ± {np.std(recall):.2f}")
print(f"Precision: {np.mean(precision):.2f} ± {np.std(precision):.2f}")
print(f"F1-score: {np.mean(f1):.2f} ± {np.std(f1):.2f}")

Accuracy: 0.73 ± 0.05
Recall: 0.62 ± 0.14
Precision: 0.68 ± 0.08
F1-score: 0.64 ± 0.09
